In [1]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

ERROR: Could not find a version that satisfies the requirement statsmodel==0.14.4 (from versions: none)
ERROR: No matching distribution found for statsmodel==0.14.4


In [22]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert
from shapely.geometry import Point
import overpy
import ast
from decimal import Decimal

from script import geolocaliser
from script import process_data
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [ ]:
# URL du fichier Excel
url_cotier = 'https://comersis.com/free-download.php?f=communes_littorales_2019.xlsx&d=littoral'

# Télécharger le fichier
response = rq.get(url_cotier)

# Lire le fichier Excel dans un DataFrame
df_cotieres = pd.read_excel(BytesIO(response.content))
df_cotieres = df_cotieres[df_cotieres['Région'] != "Départements d'Outre-Mer"]


print(df_cotieres.columns)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


Index(['Région', 'ID département', 'Nom département', 'Code Insee comm.',
       'Nom commune', 'Classement', 'Motif du classement',
       'nom de l'entité géographique dont le littoral est protégé',
       'Population', 'commune déleguée'],
      dtype='object')


C:\Users\flori\AppData\Local\Temp\ipykernel_18056\1155764729.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(df_cotieres)
C:\Users\flori\AppData\Local\Temp\ipykernel_18056\1155764729.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(df_cotieres)
C:\Users\flori\AppData\Local\Temp\ipykernel_18056\1155764729.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_18056\11557647

In [4]:
# Filtrer sur le Motif du classement
df_cotieres = df_cotieres[df_cotieres['Motif du classement'] == "Commune riveraine de la mer ou d'un océan"]
df_cotieres = df_cotieres[df_cotieres['Population'] > 1000]

print(df_cotieres.columns)
liste_cotieres = df_cotieres['Nom commune'].unique().tolist()
print(liste_cotieres[1:10])
# Statistiques par "Région" pour les communes filtrées
stats_region_filtered = df_cotieres.groupby('Région').apply(
    lambda x: pd.Series({
        'nombre_communes': x['Nom commune'].nunique(),
        'population_totale': x['Population'].sum()
    })
).reset_index()

# Afficher les résultats
print(stats_region_filtered)

Index(['Région', 'ID département', 'Nom département', 'Code Insee comm.',
       'Nom commune', 'Classement', 'Motif du classement',
       'nom de l'entité géographique dont le littoral est protégé',
       'Population', 'commune déleguée'],
      dtype='object')
['Beaulieu-sur-Mer', 'Cagnes-sur-Mer', 'Cannes', "Cap-d'Ail", 'Eze', 'Mandelieu-la-Napoule', 'Menton', 'Nice', 'Roquebrune-Cap-Martin']
                       Région  nombre_communes  population_totale
0                    Bretagne              149             912313
1                       Corse               40             217634
2             Hauts de France               40             384693
3                   Normandie               83             659647
4          Nouvelle Aquitaine               71             503920
5                   Occitanie               27             237370
6            Pays-de-la-Loire               44             352067
7  Provence-Alpes-Côte d'Azur               52            2290433


C:\Users\flori\AppData\Local\Temp\ipykernel_18056\4085312016.py:9: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [ ]:
# Charger le fichier GeoJSON des communes de France depuis le lien GitHub
url_commune = 'https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/communes.geojson'
response_commune = rq.get(url_commune)
communes_geojson = response_commune.json()

# Charger le GeoJSON dans un GeoDataFrame
gdf_communes = gpd.GeoDataFrame.from_features(communes_geojson['features'])
gdf_communes_cotieres = gdf_communes[gdf_communes['nom'].isin(liste_cotieres)]
gdf_communes_cotieres = gdf_communes_cotieres.merge(df_cotieres[['Nom commune', 'Population']], 
                                                    left_on='nom', 
                                                    right_on='Nom commune'
                                                    ).drop(columns=['Nom commune'])
# Calculer les centroïdes pour chaque commune
gdf_communes_cotieres['centroid'] = gdf_communes_cotieres.geometry.centroid
gdf_communes_cotieres['latitude_centre'] = gdf_communes_cotieres['centroid'].y
gdf_communes_cotieres['longitude_centre'] = gdf_communes_cotieres['centroid'].x
gdf_communes_cotieres = gdf_communes_cotieres.drop(columns=['centroid'])

liste_commune = gdf_communes_cotieres['nom'].to_list()

In [8]:
# Appliquer la fonction sur chaque commune
api = overpy.Overpass()

gdf_communes_cotieres[['latitude_mairie', 'longitude_mairie']] = gdf_communes_cotieres.apply(
    lambda row: pd.Series(geolocaliser.get_townhall_coordinates(row['nom'], api)),
    axis=1
)

In [10]:
# Liste des mots-clés pour localiser les mairies
mots_cles = [
    "Mairie+de", 
    "Hotel+de+ville+de", 
    "Bureau+du+Maire+de", 
    "Salle+des+Fêtes+de", 
    "Maison+Communale+de", 
    "Centre+Administratif+de", 
    "Hôtel+de+Ville", 
    "Bâtiment+Municipal+de", 
    "Mairie+municipale+de", 
    "Maison+des+Services+de"
]

# Compléter les coordonnées manquantes
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles, 
    colonne_latitude='latitude_mairie', 
    colonne_longitude='longitude_mairie'
)

Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Bonifacio
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Bonifacio
Aucune donnée trouvée pour l'adresse : Maison+des+Services+de+Bonifacio
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Zuydcoote
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Zuydcoote
Aucune donnée trouvée pour l'adresse : Maison+des+Services+de+Zuydcoote
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Château-d'Olonne
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Sauzon
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Sauzon
Aucune donnée trouvée pour l'adresse : Maison+des+Services+de+Sauzon
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Saint-Pabu
Aucune donnée trouvée pour l'adresse : Bâtiment+Municipal+de+Saint-Pabu
Aucune donnée trouvée pour l'adresse : Maison+des+Services+de+Saint-Pabu
Aucune donnée trouvée pour l'adresse : Bureau+du+Maire+de+Saint-Laurent-du-Var
Aucun

In [11]:
# Liste des mots-clés pour localiser les plages
mots_cles_plages = [ 
    "Plage+de",
    "Plage"
]

# Compléter les coordonnées manquantes pour les plages
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_plages, 
    colonne_latitude='latitude_plage', 
    colonne_longitude='longitude_plage'
)

In [12]:
show(gdf_communes_cotieres)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_18056\717718677.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_18056\717718677.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_18056\717718677.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_18056

In [13]:
# Utilisation de la fonction avec un GeoDataFrame
api = overpy.Overpass()
import importlib

# Recharger le module geolocaliser
import script.geolocaliser
importlib.reload(script.geolocaliser)
gdf_communes_cotieres['beach_coordinates'] = gdf_communes_cotieres.apply(
    lambda row: geolocaliser.get_beach_coordinates(row['nom'], api),
    axis=1
)

In [14]:
api = overpy.Overpass()

gdf_communes_cotieres['station'] = gdf_communes_cotieres.apply(
    lambda row: geolocaliser.get_station_coordinates(row['nom'], api),
    axis=1
)

In [15]:
# Liste des mots-clés pour localiser les gares
mots_cles_gares = [ 
    "Gare+de"
]

# Compléter les coordonnées manquantes pour les pgares
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_gares, 
    colonne_latitude='latitude_gare', 
    colonne_longitude='longitude_gare'
)

In [16]:
# api = overpy.Overpass()

# gdf_communes_cotieres[['coast']] = gdf_communes_cotieres.apply(
#     lambda row: pd.Series(geolocaliser.get_coastline(row['nom'], api)),
#     axis=1
# )

In [ ]:
# Utilisation de la fonction avec un GeoDataFrame
api = overpy.Overpass()
import importlib

# Recharger le module geolocaliser
import script.process_data
importlib.reload(script.process_data)

# Appliquer la fonction sur chaque commune
api = overpy.Overpass()

gdf_communes_cotieres[['latitude_port', 'longitude_port']] = gdf_communes_cotieres.apply(
    lambda row: pd.Series(geolocaliser.get_ports(row['nom'], api)),
    axis=1
)

<module 'script.process_data' from 'c:\\Users\\flori\\Documents\\Gitlab\\projet_python\\script\\process_data.py'>

In [18]:
api = overpy.Overpass()

# Liste des mots-clés pour localiser les plages
mots_cles_ports = [ 
    "Port+de",
]

# Compléter les coordonnées manquantes pour les plages
gdf_communes_cotieres = geolocaliser.geolocaliser_mot_cle(
    gdf_communes_cotieres, 
    colonne_commune='nom', 
    colonne_geometry='geometry', 
    mots_cles=mots_cles_ports, 
    colonne_latitude='latitude_port', 
    colonne_longitude='longitude_port'
)

Aucune donnée trouvée pour l'adresse : Port+de+Saint-Pabu
Aucune donnée trouvée pour l'adresse : Port+de+Criel-sur-Mer
Aucune donnée trouvée pour l'adresse : Port+de+Urrugne


In [19]:
show(gdf_communes_cotieres)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_18056\717718677.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_18056\717718677.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_18056\717718677.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_18056

In [26]:
# Application
gdf_communes_cotieres['station'] = gdf_communes_cotieres['station'].apply(process_data.fix_coordinates_format)
gdf_communes_cotieres['beach_coordinates'] = gdf_communes_cotieres['beach_coordinates'].apply(process_data.fix_coordinates_format)

In [28]:
gdf_communes_cotieres["beach_coordinates"] = gdf_communes_cotieres.apply(
    lambda row: row["beach_coordinates"] + [(row["latitude_plage"], row["longitude_plage"])] if row["latitude_plage"] is not None and row["longitude_plage"] is not None else row["beach_coordinates"],
    axis=1
)

In [30]:
# Ajout des coordonnées à beach_coordinates
gdf_communes_cotieres["station"] = gdf_communes_cotieres.apply(
    lambda row: row["station"] + [(row["latitude_gare"], row["longitude_gare"])] if row["latitude_gare"] is not None and row["longitude_gare"] is not None else row["beach_coordinates"],
    axis=1
)

In [ ]:
gdf_communes_cotieres = gdf_communes_cotieres.drop(columns=['latitude_plage','longitude_plage','latitude_gare', 'longitude_gare','code'])

In [ ]:
show(gdf_communes_cotieres)

In [32]:
gdf_communes_cotieres.to_csv('data/communes_cotieres.csv',encoding="utf8",sep=";",index=False)

In [33]:
# URL du fichier GeoJSON des départements de France
url_departement = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"

# Récupérer le fichier GeoJSON des départements de France
response = rq.get(url_departement)
departements_geojson = response.json()

# Créer une liste pour stocker les noms des départements
departement_names = []

# Extraire les noms des départements depuis les propriétés de chaque feature
for feature in departements_geojson['features']:
    departement_name = feature['properties']['nom']
    departement_names.append(departement_name)

# Créer un DataFrame avec les noms des départements
df_departements = pd.DataFrame(departement_names, columns=['departementName'])
    
# Afficher le DataFrame
print(df_departements)

          departementName
0                   Aisne
1                    Aube
2                Calvados
3                  Cantal
4            Eure-et-Loir
..                    ...
91           Loir-et-Cher
92                Mayenne
93     Meurthe-et-Moselle
94            Deux-Sèvres
95  Territoire de Belfort

[96 rows x 1 columns]
